## Roll Options and Futures

**IB-Ruby**  provides helpers to roll existing positions into other strikes and/or other expiries


In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

List Portfoliopositions 

In [2]:
G.get_account_data
puts G.clients.last.contracts.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ DBK    │ 493081947 │   DTB    │ 20210820 │    100     │     DBKE      │  put  │   11.0 │   EUR    │
│ Option │ F      │ 493470991 │   AMEX   │ 20210820 │    100     │       F       │  put  │   15.0 │   USD    │
│ Option │ IWM    │ 481723564 │   AMEX   │ 20210716 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ TKA    │ 493078274 │   DTB    │ 20210820 │    100     │      TKA      │  put  │   10.0 │   EUR    │
│ Option │ TUI1   │ 493053943 │   DTB    │ 20210820 │    100     │      TUI      │  put  │    5.0 │   EUR    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


The _Ford_-Option will be rolled into the next month

In [3]:
ford =  G.clients.last.contracts.detect{|x| x.symbol == 'F'}
# -------------------------------------------------
rolling_contract =  ford.roll expiry: '+1m'
# -------------------------------------------------
puts rolling_contract.as_table
puts rolling_contract.legs.as_table
puts rolling_contract.combo_legs.to_human

┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ F      │ -959032286 │   AMEX   │        │    100     │               │       │        │   USD    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ F      │ 493470991 │   AMEX   │ 20210820 │    100     │       F       │  put  │   15.0 │   USD    │
│ Opti

* Option#roll takes the properties of the target option as parameters
* Thus, `:expiry, :strike, :trading_class`  might be specified
* Instead of _YYYYMM_  or _YYYYMMDD_  for `:expiry``[+/-]{n}{w|m}`may be specified, too
* The existing position is **bougth**! 

The `Option#roll`helper is designed to close a short-Options position and to open another one with slightly changed attributes.


### Rolling Futures

To roll a future is equivalent to setup a calendar-spread

In [4]:
es = Symbols::Futures.es.verify.first
puts es.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Future │ ES     │ 428520022 │  GLOBEX  │ 20210917 │     50     │      ES       │       │        │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


In [5]:
rolling_future =  es.roll expiry: "+12m"
puts rolling_future.as_table
puts rolling_future.legs.as_table
puts rolling_future.combo_legs.to_human

┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ES     │ -924032588 │  GLOBEX  │        │     50     │               │       │        │   USD    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Future │ ES     │ 428520022 │  GLOBEX  │ 20210917 │     50     │      ES       │       │        │   USD    │
│ Futu

To roll an existing long-position, the `rolling_future` has to be **sold**!

To roll an existing short-position the `rolling_future` has to be **bought**!

eg:  `G.client.last.place contract: rolling_future, order: Limit_order( size: 1, price: 45, action: :sell ) ` to roll a long-Future to the other expiry